# 🧬 ITLAS Project Setup
## IT-phase Liver Immune Atlas Scoring System

**목적:** HBV chronic infection의 IT-immunopathogenesis 분석을 위한 새로운 도구 개발

### 모듈 구성
| 모듈 | 기능 | 해결하는 약점 |
|------|------|---------------|
| IT Signature Score | IT-exclusive marker 기반 scoring | 방법론적 기여 |
| FM-GSEA | Embedding + pathway integration | mTOR/glycolysis 직접 증거 |
| Phase Classifier | IT/IA/AR/AC 예측 | 임상적 유용성 |

**데이터셋:** GSE182159 (Zhang et al., Gut 2023)

---

## 1. Google Drive 마운트 및 폴더 구조 생성

In [ ]:
from google.colab import drive
import os

# Google Drive 마운트
drive.mount('/content/drive')

# 기본 경로 설정
DRIVE_ROOT = '/content/drive/MyDrive'
PROJECT_ROOT = f'{DRIVE_ROOT}/ITLAS'

print(f"✓ Drive mounted at {DRIVE_ROOT}")

In [ ]:
# 폴더 구조 자동 생성
folders = [
    f"{PROJECT_ROOT}/data/raw",
    f"{PROJECT_ROOT}/data/processed",
    f"{PROJECT_ROOT}/data/embeddings",
    f"{PROJECT_ROOT}/itlas",
    f"{PROJECT_ROOT}/notebooks",
    f"{PROJECT_ROOT}/results/figures",
    f"{PROJECT_ROOT}/results/tables",
    f"{PROJECT_ROOT}/tests",
    f"{PROJECT_ROOT}/references",
]

for folder in folders:
    os.makedirs(folder, exist_ok=True)
    print(f"✓ Created: {folder.replace(DRIVE_ROOT, '~')}")

print("\n" + "="*50)
print("📁 ITLAS 프로젝트 폴더 구조 생성 완료!")
print("="*50)

## 2. ITLAS 패키지 초기화

In [ ]:
# __init__.py
init_content = '''"""ITLAS - IT-phase Liver Immune Atlas Scoring System

A toolkit for analyzing HBV chronic infection immunopathogenesis.

Modules:
    - signature_score: IT-exclusive marker based scoring
    - fm_gsea: Foundation model embedding + pathway integration
    - classifier: IT/IA/AR/AC phase prediction

Dataset: GSE182159 (Zhang et al., Gut 2023)
"""

__version__ = "0.1.0"
__author__ = "YoungMin"
'''

with open(f"{PROJECT_ROOT}/itlas/__init__.py", 'w') as f:
    f.write(init_content)
print("✓ Created: itlas/__init__.py")

In [ ]:
# signature_score.py
sig_content = '''"""IT Signature Score Module"""

import numpy as np
import pandas as pd
from typing import Dict, Union

IT_EXCLUSIVE_MARKERS = {
    'cluster_21': {
        'genes': ['MT-ATP6', 'KLF2', 'MALAT1', 'MT-ND4L', 'MT-CO3'],
        'logfc': [1.57, 1.74, 1.44, 0.80, 0.74],
        'cell_type': 'Mito-high/Quiescence'
    },
    'cluster_23': {
        'genes': ['MT-ATP6', 'KLF2', 'MT-ND4L', 'NACA'],
        'logfc': [1.47, 1.81, 0.85, 1.02],
        'cell_type': 'Mito-high'
    },
    'cluster_25': {
        'genes': ['CD79A', 'MS4A1', 'HLA-DRA', 'CD74'],
        'logfc': [6.05, 5.97, 5.39, 4.20],
        'cell_type': 'IT-exclusive B cell'
    }
}

NK_COLLAPSE_MARKERS = {
    'genes': ['CST7', 'SRGN', 'CXCR4', 'NR4A2', 'DUSP2'],
    'it_log2or': -5.15
}


def calculate_it_signature_score(adata, method='weighted_mean', return_components=False):
    """Calculate IT phase signature score for each cell."""
    scores = {}
    
    for cluster_name, info in IT_EXCLUSIVE_MARKERS.items():
        genes = info['genes']
        weights = np.array(info['logfc'])
        available = [g for g in genes if g in adata.var_names]
        
        if len(available) == 0:
            scores[cluster_name] = np.zeros(adata.n_obs)
            continue
        
        expr = adata[:, available].X
        if hasattr(expr, 'toarray'):
            expr = expr.toarray()
        
        avail_weights = weights[:len(available)]
        avail_weights = avail_weights / avail_weights.sum()
        scores[cluster_name] = np.average(expr, axis=1, weights=avail_weights)
    
    # NK collapse score
    nk_genes = NK_COLLAPSE_MARKERS['genes']
    available_nk = [g for g in nk_genes if g in adata.var_names]
    if len(available_nk) > 0:
        nk_expr = adata[:, available_nk].X
        if hasattr(nk_expr, 'toarray'):
            nk_expr = nk_expr.toarray()
        scores['nk_collapse'] = -np.mean(nk_expr, axis=1)
    else:
        scores['nk_collapse'] = np.zeros(adata.n_obs)
    
    # Combined score
    weights = {'cluster_21': 0.25, 'cluster_23': 0.25, 'cluster_25': 0.20, 'nk_collapse': 0.30}
    combined = np.zeros(adata.n_obs)
    for comp, w in weights.items():
        if comp in scores:
            s = scores[comp]
            z = (s - np.mean(s)) / (np.std(s) + 1e-8)
            combined += w * z
    scores['IT_signature'] = combined
    
    return scores if return_components else scores['IT_signature']


def validate_against_ground_truth(adata, stage_col='Stage'):
    """Validate IT signature score against known stage labels."""
    scores = calculate_it_signature_score(adata, return_components=True)
    results = []
    
    for stage in adata.obs[stage_col].unique():
        mask = adata.obs[stage_col] == stage
        results.append({
            'Stage': stage,
            'N_cells': mask.sum(),
            'IT_score_mean': np.mean(scores['IT_signature'][mask]),
            'IT_score_std': np.std(scores['IT_signature'][mask])
        })
    
    return pd.DataFrame(results).sort_values('IT_score_mean', ascending=False)
'''

with open(f"{PROJECT_ROOT}/itlas/signature_score.py", 'w') as f:
    f.write(sig_content)
print("✓ Created: itlas/signature_score.py")

In [ ]:
# fm_gsea.py
gsea_content = '''"""FM-GSEA: Foundation Model Gene Set Enrichment Analysis"""

import numpy as np
import pandas as pd
from typing import Dict, Optional

PATHWAYS = {
    'mTOR_signaling': {
        'genes': ['MTOR', 'RPTOR', 'RICTOR', 'AKT1', 'PIK3CA', 'TSC1', 'TSC2',
                  'RHEB', 'EIF4EBP1', 'RPS6KB1', 'ULK1', 'PTEN'],
        'relevance': 'Yu et al. 2025 - HBsAg inhibits mTOR in NK cells'
    },
    'glycolysis': {
        'genes': ['HK1', 'HK2', 'GPI', 'PFKM', 'ALDOA', 'GAPDH', 'PGK1',
                  'ENO1', 'PKM', 'LDHA', 'LDHB', 'SLC2A1', 'SLC2A3'],
        'relevance': 'NK cell metabolic dysfunction in IT phase'
    },
    'oxidative_phosphorylation': {
        'genes': ['MT-ND1', 'MT-ND2', 'MT-ND3', 'MT-ND4', 'MT-ND4L', 'MT-ND5',
                  'MT-CYB', 'MT-CO1', 'MT-CO2', 'MT-CO3', 'MT-ATP6', 'MT-ATP8'],
        'relevance': 'Mito-high clusters (21, 23) in IT phase'
    },
    'nk_cell_cytotoxicity': {
        'genes': ['PRF1', 'GZMA', 'GZMB', 'GZMK', 'GNLY', 'FASLG', 'IFNG',
                  'TNF', 'KLRK1', 'KLRD1', 'NCR1', 'NCR3', 'CD226'],
        'relevance': 'NK cell collapse signature (Cluster 15)'
    },
    'il15_signaling': {
        'genes': ['IL15', 'IL15RA', 'IL2RB', 'IL2RG', 'JAK1', 'JAK3',
                  'STAT5A', 'STAT5B', 'SYK', 'LCK'],
        'relevance': 'Yu et al. 2025 - HBsAg binds IL-15Rβ'
    },
    'b_cell_differentiation': {
        'genes': ['PAX5', 'BCL6', 'PRDM1', 'IRF4', 'XBP1', 'CD79A', 'CD79B',
                  'MS4A1', 'CD19', 'CD22', 'JCHAIN', 'MZB1'],
        'relevance': 'B->Plasma differentiation block in IT'
    }
}


def calculate_pathway_scores(adata, pathways=None, method='mean'):
    """Calculate pathway activity scores for each cell."""
    if pathways is None:
        pathways = PATHWAYS
    
    scores = {}
    for name, info in pathways.items():
        genes = info['genes']
        available = [g for g in genes if g in adata.var_names]
        
        if len(available) < 3:
            scores[name] = np.zeros(adata.n_obs)
            continue
        
        expr = adata[:, available].X
        if hasattr(expr, 'toarray'):
            expr = expr.toarray()
        scores[name] = np.mean(expr, axis=1)
    
    return pd.DataFrame(scores, index=adata.obs_names)


def compare_pathway_by_stage(adata, stage_col='Stage'):
    """Compare pathway activities across disease stages."""
    from scipy.stats import mannwhitneyu
    
    pathway_scores = calculate_pathway_scores(adata)
    pathway_scores['Stage'] = adata.obs[stage_col].values
    
    results = []
    for pathway in PATHWAYS.keys():
        row = {'Pathway': pathway}
        for stage in pathway_scores['Stage'].unique():
            mask = pathway_scores['Stage'] == stage
            row[f'{stage}_mean'] = pathway_scores.loc[mask, pathway].mean()
        
        if 'IT' in pathway_scores['Stage'].values and 'NL' in pathway_scores['Stage'].values:
            it_scores = pathway_scores.loc[pathway_scores['Stage'] == 'IT', pathway]
            nl_scores = pathway_scores.loc[pathway_scores['Stage'] == 'NL', pathway]
            _, pval = mannwhitneyu(it_scores, nl_scores, alternative='two-sided')
            row['IT_vs_NL_pval'] = pval
        
        results.append(row)
    
    return pd.DataFrame(results)
'''

with open(f"{PROJECT_ROOT}/itlas/fm_gsea.py", 'w') as f:
    f.write(gsea_content)
print("✓ Created: itlas/fm_gsea.py")

In [ ]:
# classifier.py
clf_content = '''"""Phase Classifier Module"""

import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier


class PhaseClassifier:
    """HBV infection phase classifier."""
    
    def __init__(self, model_type='random_forest'):
        self.model_type = model_type
        self.model = None
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()
    
    def _get_model(self):
        if self.model_type == 'xgboost':
            try:
                from xgboost import XGBClassifier
                return XGBClassifier(n_estimators=100, max_depth=5, random_state=42)
            except ImportError:
                self.model_type = 'random_forest'
        return RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    
    def _extract_features(self, adata):
        from .signature_score import calculate_it_signature_score
        from .fm_gsea import calculate_pathway_scores
        
        sig_scores = calculate_it_signature_score(adata, return_components=True)
        pathway_df = calculate_pathway_scores(adata)
        
        features = [np.array(list(sig_scores.values())).T, pathway_df.values]
        return np.hstack(features)
    
    def fit(self, adata, stage_col='Stage'):
        X = self._extract_features(adata)
        y = self.label_encoder.fit_transform(adata.obs[stage_col])
        X_scaled = self.scaler.fit_transform(X)
        self.model = self._get_model()
        self.model.fit(X_scaled, y)
        return self
    
    def predict(self, adata):
        X = self._extract_features(adata)
        X_scaled = self.scaler.transform(X)
        y_pred = self.model.predict(X_scaled)
        return self.label_encoder.inverse_transform(y_pred)
    
    def cross_validate(self, adata, stage_col='Stage', n_folds=5):
        X = self._extract_features(adata)
        y = self.label_encoder.fit_transform(adata.obs[stage_col])
        X_scaled = self.scaler.fit_transform(X)
        
        model = self._get_model()
        cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
        acc = cross_val_score(model, X_scaled, y, cv=cv, scoring='accuracy')
        f1 = cross_val_score(model, X_scaled, y, cv=cv, scoring='f1_weighted')
        
        return {'accuracy': f"{acc.mean():.3f}±{acc.std():.3f}",
                'f1_score': f"{f1.mean():.3f}±{f1.std():.3f}"}
'''

with open(f"{PROJECT_ROOT}/itlas/classifier.py", 'w') as f:
    f.write(clf_content)
print("✓ Created: itlas/classifier.py")

In [ ]:
# utils.py
utils_content = '''"""Utility functions for ITLAS"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

STAGE_ORDER = ['NL', 'IT', 'IA', 'AR', 'AC']
STAGE_COLORS = {'NL': '#2ecc71', 'IT': '#3498db', 'IA': '#e74c3c', 'AR': '#f39c12', 'AC': '#9b59b6'}

GENE_ID_MAP = {
    'ENSG00000265972': 'MALAT1',
    'ENSG00000117289': 'TXNIP',
    'ENSG00000127528': 'KLF2',
    'ENSG00000198899': 'MT-ATP6',
    'ENSG00000212907': 'MT-ND4L',
    'ENSG00000198938': 'MT-CO3',
}


def summarize_it_findings():
    return """
    ═══════════════════════════════════════════════════════════════
    ITLAS Key Findings: IT-phase Immunopathogenesis
    ═══════════════════════════════════════════════════════════════
    
    1. NK Cell Collapse (Cluster 15)
       - Log2OR = -5.15 in IT (극심한 감소)
       - Recovery in IA phase (Log2OR = +0.28)
    
    2. IT-Exclusive Populations
       - Cluster 21: 1,465 cells (100% IT) - Mito-high/Quiescence
       - Cluster 23: 597 cells (99.8% IT) - Mito-high  
       - Cluster 25: 117 cells (100% IT) - Naive B cell
    
    3. B→Plasma Differentiation Block
       - Cluster 25 (Naive B): IT-exclusive
       - Cluster 22 (Plasma): Depleted across all disease stages
    
    4. Metabolic Stress Signature
       - Mito genes elevated: MT-ATP6, MT-ND4L, MT-CO3
       - KLF2 upregulation: Quiescence marker
    ═══════════════════════════════════════════════════════════════
    """


def plot_stage_comparison(data, value_col, stage_col='Stage', title=''):
    fig, ax = plt.subplots(figsize=(8, 5))
    order = [s for s in STAGE_ORDER if s in data[stage_col].unique()]
    colors = [STAGE_COLORS.get(s, '#333') for s in order]
    sns.boxplot(data=data, x=stage_col, y=value_col, order=order, palette=colors, ax=ax)
    ax.set_title(title)
    plt.tight_layout()
    return fig
'''

with open(f"{PROJECT_ROOT}/itlas/utils.py", 'w') as f:
    f.write(utils_content)
print("✓ Created: itlas/utils.py")

## 3. 필수 패키지 설치

In [ ]:
%%capture
!pip install scanpy anndata xgboost scikit-learn matplotlib seaborn

In [ ]:
import scanpy as sc
import sklearn
print(f"✓ scanpy: {sc.__version__}")
print(f"✓ sklearn: {sklearn.__version__}")

## 4. ITLAS 패키지 테스트

In [ ]:
import sys
sys.path.insert(0, PROJECT_ROOT)

import itlas
print(f"✓ ITLAS v{itlas.__version__} 로드 성공!")

In [ ]:
from itlas.utils import summarize_it_findings
print(summarize_it_findings())

## 5. 데이터 파일 업로드 (선택사항)

In [ ]:
from google.colab import files
import shutil

print("📂 CSV 파일들을 업로드하세요 (cluster_enrichment_stats.csv 등)")
uploaded = files.upload()

for filename in uploaded.keys():
    dst = f"{PROJECT_ROOT}/data/processed/{filename}"
    shutil.move(f"/content/{filename}", dst)
    print(f"✓ Moved: {filename}")

## 6. 프로젝트 구조 확인

In [ ]:
import os

def print_tree(path, prefix=""):
    entries = sorted(os.listdir(path))
    for e in entries:
        full = os.path.join(path, e)
        if os.path.isdir(full):
            print(f"{prefix}📁 {e}/")
            print_tree(full, prefix + "   ")
        else:
            print(f"{prefix}📄 {e}")

print("🧬 ITLAS Project Structure")
print("="*40)
print_tree(PROJECT_ROOT)

---
## ✅ Setup 완료!

### 다음 단계:
1. `01_IT_Signature_Score.ipynb` - IT scoring 테스트
2. `02_FM_GSEA.ipynb` - Pathway analysis  
3. `03_Phase_Classifier.ipynb` - 분류기 훈련

### 참고 논문:
- Zhang C, et al. **Gut 2023** (GSE182159)
- Yu Y, et al. **Cell Death Dis. 2025** (HBsAg/mTOR axis)